In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Allergies"
cohort = "GSE184382"

# Input paths
in_trait_dir = "../../input/GEO/Allergies"
in_cohort_dir = "../../input/GEO/Allergies/GSE184382"

# Output paths
out_data_file = "../../output/preprocess/Allergies/GSE184382.csv"
out_gene_data_file = "../../output/preprocess/Allergies/gene_data/GSE184382.csv"
out_clinical_data_file = "../../output/preprocess/Allergies/clinical_data/GSE184382.csv"
json_path = "../../output/preprocess/Allergies/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os

# Check the parent directory (trait directory) in case cohort is a subdirectory
print(f"Files in trait directory {in_trait_dir}:")
trait_dir_files = os.listdir(in_trait_dir)
print(trait_dir_files)

# Search for the cohort data in the parent directory
potential_cohort_paths = []
for item in trait_dir_files:
    if cohort in item:
        potential_cohort_paths.append(os.path.join(in_trait_dir, item))

print(f"\nPotential cohort paths containing '{cohort}':")
for path in potential_cohort_paths:
    print(path)
    if os.path.isdir(path):
        print(f"  Contents: {os.listdir(path)}")

# Try to find files directly in the trait directory that might match this cohort
print("\nLooking for files in trait directory that might be relevant to this cohort:")
cohort_files = []
for file in trait_dir_files:
    if cohort in file and os.path.isfile(os.path.join(in_trait_dir, file)):
        cohort_files.append(file)
        
print(f"Found {len(cohort_files)} files for cohort {cohort}: {cohort_files}")

# If we found cohort files, use the first file that looks like a matrix or SOFT file
if cohort_files:
    # Sort files to prioritize SOFT or matrix files
    soft_files = [f for f in cohort_files if 'soft' in f.lower()]
    matrix_files = [f for f in cohort_files if 'matrix' in f.lower() or 'series' in f.lower()]
    
    if soft_files:
        soft_file = os.path.join(in_trait_dir, soft_files[0])
        print(f"Using soft file: {soft_file}")
    else:
        print("No soft file found directly.")
        soft_file = None
        
    if matrix_files:
        matrix_file = os.path.join(in_trait_dir, matrix_files[0])
        print(f"Using matrix file: {matrix_file}")
    else:
        print("No matrix file found directly.")
        # If no clear matrix file, use the first file as a fallback
        matrix_file = os.path.join(in_trait_dir, cohort_files[0])
        print(f"Using fallback file for matrix: {matrix_file}")
    
    # 2. Read the matrix file to obtain background information and sample characteristics data
    try:
        background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
        clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

        # 3. Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)

        # 4. Explicitly print out all the background information and the sample characteristics dictionary
        print("\nBackground Information:")
        print(background_info)
        print("\nSample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing file: {e}")
else:
    print(f"No files found for cohort {cohort}. Cannot proceed with preprocessing.")
    
    # Set variables to allow code to continue without errors
    background_info = "No background information available"
    clinical_data = pd.DataFrame()
    sample_characteristics_dict = {}

Files in trait directory ../../input/GEO/Allergies:
['GSE169149', 'GSE182740', 'GSE184382', 'GSE185658', 'GSE192454', 'GSE203196', 'GSE203409', 'GSE205151', 'GSE230164', 'GSE270312', 'GSE84046']

Potential cohort paths containing 'GSE184382':
../../input/GEO/Allergies/GSE184382
  Contents: []

Looking for files in trait directory that might be relevant to this cohort:
Found 0 files for cohort GSE184382: []
No files found for cohort GSE184382. Cannot proceed with preprocessing.
